# Strojenie parametrów - czyli jak znaleźć najlepszy model?

![image.png](http://snoek.ddns.net/~oliver/mysite/images/blog7_accuracyprecision_banner.png)
http://snoek.ddns.net/~oliver/mysite/the-bias-variance-tradeoff.html

In [25]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
np.random.seed(123) 

In [3]:
data=pd.read_csv('heart.csv')

In [4]:
y = np.array(data['chd'])
X = data.drop(['chd'],axis=1)

In [5]:
map_dict = {'Present': 1, 'Absent':0}
X['famhist'] = X['famhist'].map(map_dict)
X.head()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age
0,160,12.00,5.73,23.11,1,49,25.30,97.20,52
1,144,0.01,4.41,28.61,0,55,28.87,2.06,63
2,118,0.08,3.48,32.28,1,52,29.14,3.81,46
3,170,7.50,6.41,38.03,1,51,31.99,24.26,58
4,134,13.60,3.50,27.78,1,60,25.99,57.34,49


### Podział na train, test

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.2)

### Drzewo decyzyjne

In [7]:
from sklearn.tree import DecisionTreeClassifier
#parametry domyślne
tree_model= DecisionTreeClassifier()

In [8]:
tree_model

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [9]:
tree_model.fit(X_train, y_train);

In [10]:
tree_model.score(X_test, y_test)

0.6344086021505376

### Kroswalidacja

In [11]:
import sklearn
from sklearn.model_selection import cross_val_score
tree_model= DecisionTreeClassifier()
results=cross_val_score(tree_model, X, y) # można zdefiniować: scoring='roc_auc'
# uwaga: tutaj nie strojono parametrów więc można użyć całego zbioru
print(np.mean(results), np.std(results))

0.6644693782141188 0.023197947221853043


In [12]:
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])


![image.png](https://mk0analyticsindf35n9.kinstacdn.com/wp-content/uploads/2018/06/both.png)
https://analyticsindiamag.com/why-is-random-search-better-than-grid-search-for-machine-learning/

### Grid Search

In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
max_depth=[3, 5, 6]
criterion=["gini","entrophy"]
ccp_alpha=[0, 0.05]
param_grid = dict(max_depth=max_depth,criterion=criterion, ccp_alpha=ccp_alpha)

In [15]:
tree_model= DecisionTreeClassifier()
grid = GridSearchCV(estimator=tree_model, param_grid=param_grid, cv = 3, n_jobs=-1)

grid_result = grid.fit(X_train, y_train) #tutaj lepiej zastosować tylko trainset

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.680217 using {'ccp_alpha': 0, 'criterion': 'gini', 'max_depth': 3}


In [16]:
std_test_scores = grid_result.cv_results_['std_test_score']
mean_test_scores = grid_result.cv_results_['mean_test_score']
parameters = grid_result.cv_results_['params']

for mean_test, std_test, params in zip(mean_test_scores, std_test_scores, parameters):
    print('Parameters : {}'.format(params))
    print('Mean test score: {:.8f}'.format(mean_test))
    print('Standard Deviation (test) : {:.8f}'.format(std_test))
    print()

print('Best parameters : {}'.format(grid_result.best_params_))
print('Best accuracy score : {:.8f}'.format(grid_result.best_score_))


Parameters : {'ccp_alpha': 0, 'criterion': 'gini', 'max_depth': 3}
Mean test score: 0.68021680
Standard Deviation (test) : 0.04267755

Parameters : {'ccp_alpha': 0, 'criterion': 'gini', 'max_depth': 5}
Mean test score: 0.66666667
Standard Deviation (test) : 0.02893517

Parameters : {'ccp_alpha': 0, 'criterion': 'gini', 'max_depth': 6}
Mean test score: 0.65582656
Standard Deviation (test) : 0.04419920

Parameters : {'ccp_alpha': 0, 'criterion': 'entrophy', 'max_depth': 3}
Mean test score: nan
Standard Deviation (test) : nan

Parameters : {'ccp_alpha': 0, 'criterion': 'entrophy', 'max_depth': 5}
Mean test score: nan
Standard Deviation (test) : nan

Parameters : {'ccp_alpha': 0, 'criterion': 'entrophy', 'max_depth': 6}
Mean test score: nan
Standard Deviation (test) : nan

Parameters : {'ccp_alpha': 0.05, 'criterion': 'gini', 'max_depth': 3}
Mean test score: 0.65853659
Standard Deviation (test) : 0.00663818

Parameters : {'ccp_alpha': 0.05, 'criterion': 'gini', 'max_depth': 5}
Mean test sc

In [17]:
best_model=grid_result.best_estimator_
best_model.score(X_test, y_test)

0.6666666666666666

przykład wizualizacji:
    https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html

### Random Search 

In [18]:
from sklearn.model_selection import RandomizedSearchCV

In [19]:
random = RandomizedSearchCV(estimator=tree_model, param_distributions=param_grid, cv = 3, n_jobs=-1)

random_result = random.fit(X_train, y_train)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))

Best: 0.680217 using {'max_depth': 3, 'criterion': 'gini', 'ccp_alpha': 0}


In [20]:
best_model=random_result.best_estimator_
best_model.score(X_test, y_test)

0.6666666666666666

- Zdefiniowanie rozkładów

In [21]:
from scipy.stats import poisson,expon
param_grid = {'ccp_alpha': expon(0.08),
             'max_depth': poisson(5)}

In [22]:
random = RandomizedSearchCV(estimator=tree_model, param_distributions=param_grid, cv = 3, n_jobs=-1, random_state=123)

random_result = random.fit(X_train, y_train)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))

Best: 0.655827 using {'ccp_alpha': 1.2722721434811057, 'max_depth': 7}


https://ksopyla.com/machine-learning/grid-random-search-scikit-learn-dobor-parametrow/

## Bayes optimization

In [23]:
#!pip install scikit-optimize

In [26]:
from skopt import BayesSearchCV
from sklearn.svm import SVC

# log-uniform: understand as search over p = exp(x) by varying x
opt = BayesSearchCV(
    SVC(),
    {
        'C': (1e-6, 1e+6, 'log-uniform'),
        'degree': (1, 8),  # integer valued parameter
        'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
    },
    n_iter=5, #generalnie powinno być więcej, ale wtedy długo trwa
    cv=3
)

opt.fit(X_train, y_train)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_test, y_test))

val. score: 0.7127371273712737
test score: 0.6881720430107527


https://scikit-optimize.github.io/stable/auto_examples/sklearn-gridsearchcv-replacement.html

## Jak robić strojenie parametrów dla różnych modeli?
- http://www.davidsbatista.net/blog/2018/02/23/model_optimization/
- https://stackoverflow.com/questions/50265993/alternate-different-models-in-pipeline-for-gridsearchcv

## automl from sklearn
(działa tylko na Linuxie - rozwiązanie: korzystać z Google Colaboratory)

In [150]:
import autosklearn.classification
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)
y_pred = automl.predict(X_test)
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_pred))

ModuleNotFoundError: No module named 'autosklearn'

### TPOT

![image.png](https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1537396029/output_2_0_d7uh0v.png)
https://www.datacamp.com/community/tutorials/tpot-machine-learning-python

In [ ]:
# ! pip install tpot

In [27]:
from tpot import TPOTClassifier
#from tpot import TPOTRegressor

tpot = TPOTClassifier(generations=5,verbosity=2)

tpot.fit(X_train, y_train)

Generation 1 - Current best internal CV score: 0.7209922251018142
Generation 2 - Current best internal CV score: 0.7262865605331358
Generation 3 - Current best internal CV score: 0.7290633098852276
Generation 4 - Current best internal CV score: 0.7290633098852276
Generation 5 - Current best internal CV score: 0.734394668641244

Best pipeline: GaussianNB(PCA(ExtraTreesClassifier(StandardScaler(input_matrix), bootstrap=True, criterion=entropy, max_features=0.6500000000000001, min_samples_leaf=18, min_samples_split=17, n_estimators=100), iterated_power=8, svd_solver=randomized))


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=5,
               max_eval_time_mins=5, max_time_mins=None, memory=None,
               mutation_rate=0.9, n_jobs=1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=100,
               random_state=None, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

In [32]:
tpot.score(X_test, y_test)

0.7634408602150538

## Feature importance

In [33]:
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [34]:
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]


plt.figure()
plt.title("Feature importance")
plt.bar(X.columns, importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(rotation=45)
plt.show()

## Selekcja zmiennych

### SelectKBest (na podstawie algorytmu ANOVA)

In [53]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
tree_model=DecisionTreeClassifier()

pipe = Pipeline([
    ('select', SelectKBest()),
    ('model', tree_model)])

k=[5,6]
max_depth=[3, 5, 6]
criterion=["gini","entrophy"]

# uwaga: gdy podajemy parametry do strojenia gdy mamy pipeline to trzeba w nazwach kluczy podać nazwę_danego_etapu__ 
# (poprzedzoną dwoma podkreślnikami)
param_grid = {"model__max_depth": max_depth, "model__criterion": criterion, "select__k": k}

search = GridSearchCV(pipe, param_grid, cv=5).fit(X_train, y_train)

In [54]:
search.score(X_test, y_test)

0.7204301075268817

### Recursive Feature Elimination

In [55]:
from sklearn.feature_selection import RFE
estimator = tree_model
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train) 

In [56]:
print(selector.support_)
selector.ranking_

[False False  True False False False  True False  True]


array([6, 3, 1, 4, 7, 2, 1, 5, 1])

### Boruta

In [57]:
#!pip install boruta

In [58]:
from boruta import BorutaPy
# for classification only (we need to convert pd.DataFrame to np.array)

In [65]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

feat_selector.fit(X_train.values, y_train)
feat_selector.support_
feat_selector.ranking_

# wybór odpowiednich zmiennych ze zbioru testowego
X_filtered = feat_selector.transform(X_test.values)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	9
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	4
Tentative: 	3
Rejected: 	2
Iteration: 	9 / 100
Confirmed: 	4
Tentative: 	3
Rejected: 	2
Iteration: 	10 / 100
Confirmed: 	4
Tentative: 	3
Rejected: 	2
Iteration: 	11 / 100
Confirmed: 	4
Tentative: 	3
Rejected: 	2
Iteration: 	12 / 100
Confirmed: 	4
Tentative: 	3
Rejected: 	2
Iteration: 	13 / 100
Confirmed: 	4
Tentative: 	3
Rejected: 	2
Iteration: 	14 / 100
Confirmed: 	4
Tentative: 	3
Rejected: 	2
Iteration: 	15 / 100
Confirmed: 	4
Tentative: 	3
Rejected: 	2
Iteration: 	16 / 100
Confirmed: 	4
Tentative: 	2
Rejected: 	3
Iteration: 	17 / 

### Problem niezbalansowania klas

In [60]:
#!pip install imbalanced-learn

In [66]:
import imblearn
from imblearn.over_sampling import SMOTE

In [67]:
print(X_train.shape)
pd.Series(y_train).value_counts()
# akurat tutaj ten balans jest w miarę ok, ale dla celów demonstracyjnych zrobimy na nim upsample

(369, 9)


0    242
1    127
dtype: int64

In [68]:
# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(X_train, y_train)

In [69]:
print(X.shape)
pd.Series(y).value_counts()

(484, 9)


1    242
0    242
dtype: int64